In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import random
import warnings
warnings.filterwarnings('ignore')

# Importing Datasets

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Data Exploration

In [3]:
# train_df = train_df.loc[:len(train_df)/10, :]
train_df.shape

(200000, 202)

In [4]:
train_df.head(3)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965


In [5]:
train_df.drop(['target', 'ID_code'], axis=1).sum(axis=1)

0         1456.3182
1         1415.3636
2         1240.8966
3         1288.2319
4         1354.2310
5         1272.3216
6         1509.4490
7         1438.5083
8         1369.7375
9         1303.1155
10        1379.3592
11        1240.6090
12        1435.2787
13        1387.4958
14        1474.2376
15        1123.0991
16        1372.0025
17        1403.1321
18        1390.4613
19        1265.4414
20        1353.1670
21        1335.9404
22        1331.0470
23        1393.8443
24        1405.6975
25        1282.8065
26        1427.6995
27        1356.3547
28        1305.3099
29        1316.2279
            ...    
199970    1375.8126
199971    1413.7639
199972    1354.3063
199973    1423.5793
199974    1414.7987
199975    1357.0326
199976    1332.4349
199977    1377.1981
199978    1385.4342
199979    1352.4059
199980    1270.1532
199981    1383.4933
199982    1423.6153
199983    1418.6171
199984    1340.3993
199985    1482.8420
199986    1422.1546
199987    1311.7743
199988    1328.5361


In [6]:
train_df.drop(['target', 'ID_code'], axis=1).mean(axis=1)

0         7.281591
1         7.076818
2         6.204483
3         6.441159
4         6.771155
5         6.361608
6         7.547245
7         7.192542
8         6.848687
9         6.515578
10        6.896796
11        6.203045
12        7.176393
13        6.937479
14        7.371188
15        5.615496
16        6.860012
17        7.015660
18        6.952306
19        6.327207
20        6.765835
21        6.679702
22        6.655235
23        6.969222
24        7.028488
25        6.414032
26        7.138497
27        6.781773
28        6.526550
29        6.581140
            ...   
199970    6.879063
199971    7.068820
199972    6.771531
199973    7.117897
199974    7.073994
199975    6.785163
199976    6.662174
199977    6.885990
199978    6.927171
199979    6.762030
199980    6.350766
199981    6.917467
199982    7.118076
199983    7.093085
199984    6.701997
199985    7.414210
199986    7.110773
199987    6.558871
199988    6.642681
199989    7.531528
199990    6.877515
199991    6.

In [7]:
train_df.drop(['target', 'ID_code'], axis=1).std(axis=1)

0          9.331540
1         10.336130
2          8.753387
3          9.594064
4         11.287122
5          9.313012
6          9.246130
7          9.162558
8          9.837520
9          9.943238
10         9.479191
11         8.982570
12         9.653273
13         9.094237
14         9.983810
15         9.392711
16         9.143804
17         9.207644
18         9.788900
19        10.316098
20         9.250342
21        10.076662
22        10.697809
23         9.350851
24        10.030591
25        10.533132
26        10.316578
27        10.268086
28         9.131534
29         9.206875
            ...    
199970     8.990195
199971     9.898794
199972     9.895548
199973     9.838705
199974     9.408184
199975     9.375664
199976     8.793638
199977     9.774247
199978     9.293264
199979    10.286800
199980     9.555700
199981     9.601411
199982    10.630775
199983     8.793288
199984     9.964289
199985     9.991038
199986     9.216567
199987     9.723047
199988    10.367365


# Feature Engineering

### Merging df_train and test first

In [8]:
y_train = train_df["target"].values
train_id = train_df['ID_code'].values
test_id = test_df['ID_code'].values

df_merge = pd.concat([train_df.drop('target',axis=1),test_df],axis=0)
df_merge.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [9]:
df_merge['sum'] = df_merge.drop(['ID_code'], axis=1).sum(axis=1).values
df_merge['mean'] = df_merge.drop(['ID_code'], axis=1).mean(axis=1).values
df_merge['std'] = df_merge.drop(['ID_code'], axis=1).std(axis=1).values
df_merge['min'] = df_merge.drop(['ID_code'], axis=1).min(axis=1).values
df_merge['max'] = df_merge.drop(['ID_code'], axis=1).max(axis=1).values
df_merge['skew'] = df_merge.drop(['ID_code'], axis=1).skew(axis=1).values
df_merge['kurt'] = df_merge.drop(['ID_code'], axis=1).kurtosis(axis=1).values
df_merge['med'] = df_merge.drop(['ID_code'], axis=1).median(axis=1).values

In [10]:
df_merge.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_198,var_199,sum,mean,std,min,max,skew,kurt,med
0,train_0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,12.7803,-1.0914,1456.3182,14.490728,102.374526,-21.4494,1456.3182,9.950662,99.118715,7.3746
1,train_1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,18.3560,1.9518,1415.3636,14.083220,99.617800,-47.3797,1415.3636,9.922921,98.752495,7.5109
2,train_2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,14.7222,0.3965,1240.8966,12.347230,87.307085,-22.4038,1240.8966,9.936074,98.923499,6.2846
3,train_3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,17.9697,-8.9996,1288.2319,12.818228,90.689196,-35.1659,1288.2319,9.923063,98.753287,7.0364
4,train_4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,17.9974,-8.8104,1354.2310,13.474935,95.468663,-65.4863,1354.2310,9.886546,98.282259,7.0642


### Restore datraframes df_train and df_test

In [11]:
train_df = df_merge[:len(train_df)]
train_df["target"] = y_train.tolist()
test_df = df_merge[len(train_df):]

train_df['ID_code'] = train_id
test_df['ID_code'] = test_id

In [12]:
from sklearn.ensemble import RandomForestClassifier

X = train_df.drop(['target', 'ID_code'], axis=1).values
X_test = test_df.drop(['ID_code'], axis=1).values
y = train_df.target.values

n_models = 1
preds = []
for n in range(n_models):
    print(f'Training RF model {n+1}...')
    clf = RandomForestClassifier(max_features='auto', oob_score=True, random_state=0, n_jobs=-1,
                                n_estimators=200, max_depth=15, verbose=1
                                )
    clf.fit(X, y)
    preds.append(clf.predict_proba(X_test))
print('Ended training!')

Training RF model 1...
Ended training!


In [13]:
preds

[array([[0.87483584, 0.12516416],
        [0.88005526, 0.11994474],
        [0.86886983, 0.13113017],
        ...,
        [0.95239964, 0.04760036],
        [0.87048783, 0.12951217],
        [0.90650155, 0.09349845]])]

In [14]:
np.mean(preds, axis=0)[:, 1]

array([0.12516416, 0.11994474, 0.13113017, ..., 0.04760036, 0.12951217,
       0.09349845])

# Submit Answer

In [16]:
import time  
test_df['target'] = np.mean(preds, axis=0)[:, 1]

submission_string = 'RF_' + str(n_models)+ 'models_' + time.strftime('%Y-%m-%d %H:%M:%S') + '.csv'
submission_string = submission_string.replace(':', '_')
test_df.loc[:, ['ID_code', 'target']].to_csv(submission_string, index=False)